In [5]:
from azureml.core import Workspace
from azureml.core import Model

# Create or retrieve an existing Azure ML workspace
#ws = Workspace.get(name='myworkspace',
subscription_id='097bdd2d-ace6-4c0e-9deb-5a36490948d5',
resource_group='ai_and_ml_experiments',
location='eastus'
# )

# used tupples from above
ws = Workspace.create(name='ai_and_ml_experiments_ws',
                      subscription_id=subscription_id[0],  # Extract string from tuple
                      resource_group=resource_group[0],    # Extract string from tuple
                      location='eastus')

# Write configuration to the workspace config file
ws.write_config(path='.azureml')

import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network with one hidden layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer (784 input features)
        self.fc2 = nn.Linear(128, 10)   # Output layer (10 classes)
        self.relu = nn.ReLU()           # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply ReLU after the first layer
        x = self.fc2(x)             # Output layer
        return x

# Initialize the neural network
model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

from torchvision import datasets, transforms

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(inputs.view(-1, 784))  # Flatten input
        loss = criterion(outputs, labels)      # Compute loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'simple_nn.pth')

# Register the model in Azure
model = Model.register(workspace=ws, model_path='simple_nn.pth', model_name='simple_nn')

# Deploying as a web service requires more steps such as creating a scoring script and environment.

Deploying StorageAccount with name aiandmlestorageacbcd69e7.
Deploying Workspace with name ai_and_ml_experiments_ws.
Deploying AppInsights with name aiandmleinsights9bc82bbd.
Deployed AppInsights with name aiandmleinsights9bc82bbd. Took 23.9 seconds.
Deployed StorageAccount with name aiandmlestorageacbcd69e7. Took 22.78 seconds.
Deployed Workspace with name ai_and_ml_experiments_ws. Took 16.8 seconds.
Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 25753919.54it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 657838.48it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 6338574.07it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



Epoch 1, Loss: 0.8659879893779755
Epoch 2, Loss: 0.3768385720769564
Epoch 3, Loss: 0.32350854086875913
Epoch 4, Loss: 0.29354297189911205
Epoch 5, Loss: 0.2711963312208652
Registering model simple_nn
